<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [10]:
%%writefile get_data.py

from pathlib import Path

import os
import requests
import zipfile

# Setup path to data folder
data_path = Path("data/")
food_data_path = data_path / "pizza_steak_sushi"

# Check if the food data already exists. If not, download it.
if food_data_path.is_dir():
    print(f"{food_data_path} already exists. Downloading skipped!")
    exit(0)
else:
    print(f"Did not find {food_data_path}. Creating one...")
    food_data_path.mkdir(parents=True, exist_ok=True)
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        print("Downloading food data...")
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        f.write(request.content)
    
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping food data...")
        zip_ref.extractall(food_data_path)

# Remove the zip file after downloading and extracting
print("Removing zip file: pizza_steak_sushi.zip")
os.remove(data_path / "pizza_steak_sushi.zip")

print(f"Food data successfully downloaded and extracted to {food_data_path}")

Overwriting get_data.py


In [11]:
# Example running of get_data.py
!python get_data.py

data\pizza_steak_sushi already exists. Downloading skipped!


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [5]:
%%writefile train.py
# YOUR CODE HERE

"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import argparse
import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Initializing argument parser and adding command-line arguments
parser = argparse.ArgumentParser(description="The programs takes in hyper-parameters from CLI and train and evaluate the TinyVGG model.")
parser.add_argument("--epochs", default=5, type=int, help="Number of epochs to train the model for.")
parser.add_argument("--data", default="./data", type=str, help="Location where training and testing data is stored.")
parser.add_argument("--batch_size", default=16, type=int, help="Batch size to train the model on for single iteration.")
parser.add_argument("--lr", default=0.001, type=float, help="Learning rate for the optimizer.")
parser.add_argument("--num_filters", default=10, type=int, help="Number of filters (kernels) in a single convolutional layer.")
args = parser.parse_args()
data_base_dir = args.data

if data_base_dir == "./data" or not os.path.isdir(data_base_dir):
    print("Please specify the target dataset.")
    exit(0)

# Setup hyperparameters
NUM_EPOCHS = args.epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.num_filters
LEARNING_RATE = args.lr

# Setup directories
train_dir = data_base_dir + "/train"
test_dir = data_base_dir + "/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting train.py


In [7]:
# Example running of train.py
!python train.py --epochs 20 --batch_size 32 --lr 0.003 --data "data/pizza_steak_sushi"

^C


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [4]:
%%writefile predict.py

# YOUR CODE HERE
import model_builder
import sys
import torch
import torchvision

from torchvision import transforms

if len(sys.argv) < 2:
    print("Please an image location to run inference on.")
    exit(0)

MODEL_PATH = "./models/05_going_modular_script_mode_tinyvgg_model.pth"
CLASSES = ['pizza', 'steak', 'sushi']

device = "cuda" if torch.cuda.is_available() else "cpu"

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=3
).to(device)

# Load the state of the model (trained parameters)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# Load in custom image and convert the tensor values to float32
custom_image = torchvision.io.read_image(str(sys.argv[1])).type(torch.float32)

# Divide the image pixel values by 255 to get them between [0, 1]
custom_image = custom_image / 255. 

# Create transform pipleine to resize image
custom_image_transform = transforms.Compose([
    transforms.Resize((64, 64)),
])

# Transform target image
custom_image_transformed = custom_image_transform(custom_image).unsqueeze(dim=0).to(device)

with torch.inference_mode():
    custom_image_pred = model(custom_image_transformed)
    custom_image_pred_probs = torch.softmax(custom_image_pred, dim=1)
    print(f"Prediction probabilities: {custom_image_pred_probs}")

    # Convert prediction probabilities -> prediction labels
    custom_image_pred_label = torch.argmax(custom_image_pred_probs, dim=1).cpu().item()
    pred_label_class = CLASSES[custom_image_pred_label]
    print(pred_label_class)


Overwriting predict.py


In [5]:
# Example running of predict.py 
!python predict.py data/pizza_steak_sushi/test/sushi/175783.jpg

Prediction probabilities: tensor([[0.9453, 0.0240, 0.0307]], device='cuda:0')
pizza
